In [1]:
import json
import re
from langchain.chat_models  import AzureChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from backend.src.utils import MySQLAgent

import google.generativeai as genai
import google.ai.generativelanguage as glm
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

c:\Users\sean.chang\AppData\Local\anaconda3\envs\credit_invest\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
def read_config(path):
    try:
        with open(path, 'r') as file:
            configs = json.load(file)

        return configs
    except FileNotFoundError:
        print(f"The file {path} was not found.")
    except json.JSONDecodeError:
        print(f"Error decoding JSON from the file {path}.")

configs = read_config(path="./backend/.env/connections.json")
job_configs = configs["CREDITREPORT"]['Crawler_mysql_conn_info']
sql_agent = MySQLAgent(job_configs)

In [39]:
sql_agent

In [37]:
company_id = 83387850
query = f"""
SELECT judgement_date, judgement_no, subjectkey, judgement_text01
FROM judgement_result
WHERE business_accounting_no = '{company_id}'
  AND judgement_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 5 YEAR);
"""
df_judgement = sql_agent.read_table(query=query)

In [38]:
df_judgement = df_judgement.drop_duplicates()
df_judgement

,judgement_date,judgement_no,subjectkey,judgement_text01
0,2020-10-07,最高法院 109 年度台上字第 1064 號民事裁定,請求給付貨款,最高法院民事裁定 109年度台上字第1064號\n上 訴 人 傳福企業股份...


In [35]:
for _, row in df_judgement.iterrows():
    print(row['judgement_no'])

最高法院 109 年度台上字第 1064 號民事裁定


## Statistic

querykey 

In [5]:
df_judgement.groupby('subjectkey').size().to_dict()

{'請求給付貨款': 1}

data clean

In [6]:
df_judgement.iloc[0].judgement_text01

'最高法院民事裁定\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000 109年度台上字第1064號\n上\u3000訴\u3000人\u3000傳福企業股份有限公司\n法定代理人\u3000劉文忠\n訴訟代理人\u3000謝憲杰律師\n\u3000\u3000\u3000\u3000\u3000\u3000邰怡瑄律師\n\u3000\u3000\u3000\u3000\u3000\u3000陳清怡律師\n被 上訴 人\u3000三芳化學工業股份有限公司\n法定代理人\u3000林孟經\n訴訟代理人\u3000王伊忱律師\n上列當事人間請求給付貨款事件，上訴人對於中華民國107 年11\n月7日臺灣高等法院第二審判決（106年度上字第1643號），提起\n上訴，本院裁定如下：\n    主  文\n上訴駁回。\n第三審訴訟費用由上訴人負擔。\n    理  由\n按上訴第三審法院，非以原判決違背法令為理由，不得為之。又\n提起上訴，上訴狀內應記載上訴理由，表明原判決所違背之法令\n及其具體內容，暨依訴訟資料合於該違背法令之具體事實，其依\n民事訴訟法第469條之1規定提起上訴者，並應具體敘述為從事法\n之續造、確保裁判之一致性或其他所涉及之法律見解具有原則上\n重要性之理由。同法第467條、第470條第2 項定有明文。而依同\n法第468 條規定，判決不適用法規或適用不當者，為違背法令；\n依同法第469 條規定，判決有該條所列各款情形之一者，為當然\n違背法令。是當事人提起上訴，如以同法第469 條所列各款情形\n為理由時，其上訴狀或理由書應表明該判決有合於各該條款規定\n情形之具體內容，及係依何訴訟資料合於該違背法令之具體事實\n。如依同法第468 條規定，以原判決有不適用法規或適用法規不\n當為理由時，其上訴狀或理由書應表明該判決所違背之法令條項\n，或有關解釋字號，或成文法以外之習慣或法理等及其具體內容\n，暨係依何訴訟資料合於該違背法令之具體事實，並具體敘述為\n從事法之續造、確保裁判之一致性或其他所涉及之法律見解具有\n原則上重要性之理由。上訴狀或理由書如未依上述方法表明，或\n其所表明者與上開法條規定不合時，即難認為已合法表明上訴理\n由，其上訴自非合法。本件上訴人對於原判決提起上訴，雖以該\n判

In [7]:
def clean_data(data):
    # Remove specific unicode characters
    cleaned_data = re.sub(r'\u3000', '', data)
    
    # Replace multiple spaces with a single space
    cleaned_data = re.sub(r'\s{2,}', ' ', cleaned_data)
    
    # Remove leading and trailing spaces on each line
    cleaned_data = re.sub(r'^\s+|\s+$', '', cleaned_data, flags=re.MULTILINE)
    
    # Remove redundant newlines
    cleaned_data = re.sub(r'\n\s*\n', '\n', cleaned_data)

    cleaned_data = re.sub(r'\n', '', cleaned_data)

    cleaned_data = re.sub(r'\s', '', cleaned_data)
    
    return cleaned_data

def extract_section(data, start_keyword, end_keyword):
    # Use regex to find the text between start_keyword and end_keyword
    pattern = re.compile(f'{start_keyword}(.*?){end_keyword}', re.DOTALL)
    match = pattern.search(data)
    if match:
        return match.group(1).strip()
    else:
        return None

In [8]:
clean_test = clean_data(df_judgement.iloc[0].judgement_text01)
clean_test

'最高法院民事裁定109年度台上字第1064號上訴人傳福企業股份有限公司法定代理人劉文忠訴訟代理人謝憲杰律師邰怡瑄律師陳清怡律師被上訴人三芳化學工業股份有限公司法定代理人林孟經訴訟代理人王伊忱律師上列當事人間請求給付貨款事件，上訴人對於中華民國107年11月7日臺灣高等法院第二審判決（106年度上字第1643號），提起上訴，本院裁定如下：主文上訴駁回。第三審訴訟費用由上訴人負擔。理由按上訴第三審法院，非以原判決違背法令為理由，不得為之。又提起上訴，上訴狀內應記載上訴理由，表明原判決所違背之法令及其具體內容，暨依訴訟資料合於該違背法令之具體事實，其依民事訴訟法第469條之1規定提起上訴者，並應具體敘述為從事法之續造、確保裁判之一致性或其他所涉及之法律見解具有原則上重要性之理由。同法第467條、第470條第2項定有明文。而依同法第468條規定，判決不適用法規或適用不當者，為違背法令；依同法第469條規定，判決有該條所列各款情形之一者，為當然違背法令。是當事人提起上訴，如以同法第469條所列各款情形為理由時，其上訴狀或理由書應表明該判決有合於各該條款規定情形之具體內容，及係依何訴訟資料合於該違背法令之具體事實。如依同法第468條規定，以原判決有不適用法規或適用法規不當為理由時，其上訴狀或理由書應表明該判決所違背之法令條項，或有關解釋字號，或成文法以外之習慣或法理等及其具體內容，暨係依何訴訟資料合於該違背法令之具體事實，並具體敘述為從事法之續造、確保裁判之一致性或其他所涉及之法律見解具有原則上重要性之理由。上訴狀或理由書如未依上述方法表明，或其所表明者與上開法條規定不合時，即難認為已合法表明上訴理由，其上訴自非合法。本件上訴人對於原判決提起上訴，雖以該判決違背法令為由，惟核其上訴理由狀所載內容，係就原審取捨證據、認定事實之職權行使，及依職權解釋契約所論斷：上訴人於民國103年9月至11月間向被上訴人訂購如原判決附表（下稱附表）一、二所示單價、數量之PU合成皮，約定該PU合成皮須符合「PUMA物性標準」；被上訴人其後交付上訴人如附表一「交貨數量」欄所示之PU合成皮，並完成附表二「被上訴人主張完工數量」欄所示之PU合成皮。兩造各自取樣之PU合成皮經臺灣檢驗科技股份有限公司鞋材物性實驗室（下稱SGS）鑑定結果，「磨耗超過100次，均無表皮破損現象產生」、「並無觀察到⒈樣品表面漆皮

In [9]:
start_keyword = "主文"
end_keyword = "事實"

result = extract_section(clean_test, start_keyword, end_keyword)
result

'上訴駁回。第三審訴訟費用由上訴人負擔。理由按上訴第三審法院，非以原判決違背法令為理由，不得為之。又提起上訴，上訴狀內應記載上訴理由，表明原判決所違背之法令及其具體內容，暨依訴訟資料合於該違背法令之具體'

model

In [12]:
import os
configs = read_config(path="./backend/.env/configs.json")
api_key =configs['g_key']
os.environ["GOOGLE_API_KEY"] = api_key
genai.configure(api_key=api_key)

In [13]:
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    convert_system_message_to_human=True,
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
)

In [23]:
# 加上該公司在主文裡面扮演的腳色，或是他受到什麼影響

messages = [
    SystemMessage(content="你是一個專業的摘要統整AI助手，針對我提出的問題使用繁體中文回覆。"),
    HumanMessage(content=f"""
    #### INSTRUCTION: 仔細閱讀 REF 的內容，提供我以下內容:
                1. REF 的摘要
                2. COMPANY 公司在其中扮演的腳色
                3. COMPANY 公司在其中受到的影響
                若沒有找到滿足我需求的內容則說沒有，不要亂作答不然外婆會難過。
    #### REF: {clean_test}
    #### COMPANY: 三芳化學
""")
]

result = llm(messages)

c:\Users\sean.chang\AppData\Local\anaconda3\envs\credit_invest\lib\site-packages\langchain_google_genai\chat_models.py:345: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


In [24]:
result.content

'1. 摘要：\n   最高法院民事裁定109年度台上字第1064號上訴人傳福企業股份有限公司對被上訴人三芳化學工業股份有限公司請求給付貨款。上訴人認為原判決違背法令，但在上訴理由狀中未具體說明違背法令的部分，因此上訴為不合法。\n\n2. 公司在其中扮演的腳色：\n   被上訴人三芳化學工業股份有限公司。\n\n3. 公司在其中受到的影響：\n   由於上訴人上訴不合法，因此三芳化學工業股份有限公司不受影響。'

## Data check

In [23]:
query = """
SELECT *
FROM judgement_result
WHERE judgement_no = '臺灣桃園地方法院 112 年度訴字第 2129 號民事裁定'
ORDER BY autoid
"""
df_test = sql_agent.read_table(query=query)

In [29]:
df_test_2 = df_test.drop(['creation_dt','update_dt', 'autoid'], axis=1)

In [35]:
print(f'去重複前資料筆數:{df_test.shape[0]}')
print(f'去重複後資料筆數:{df_test_2.drop_duplicates().shape[0]}')

去重複前資料筆數:10
去重複後資料筆數:5


In [30]:
df_test_2.drop_duplicates()

,business_accounting_no,companyname,judgement_class,judgement_date,judgement_no,querykey,subjectkey,judgement_text01,judgement_hyperlink,datasource_num_id
0,04226091,台燿科技股份有限公司,判決書,2023-12-07,臺灣桃園地方法院 112 年度訴字第 2129 號民事裁定,給付貨款,給付貨款,,https://judgment.judicial.gov.tw/FJUD/data.asp...,26
1,04226091,台燿科技股份有限公司,判決書,2023-11-16,臺灣桃園地方法院 112 年度訴字第 2129 號民事裁定,給付貨款,給付貨款,,https://judgment.judicial.gov.tw/FJUD/data.asp...,26
2,04226091,台燿科技股份有限公司,判決書,2023-10-26,臺灣桃園地方法院 112 年度訴字第 2129 號民事裁定,給付貨款,給付貨款,,https://judgment.judicial.gov.tw/FJUD/data.asp...,26
3,04226091,台燿科技股份有限公司,判決書,2023-11-16,臺灣桃園地方法院 112 年度訴字第 2129 號民事裁定,支付命令,給付貨款,,https://judgment.judicial.gov.tw/FJUD/data.asp...,26
4,04226091,台燿科技股份有限公司,判決書,2023-10-26,臺灣桃園地方法院 112 年度訴字第 2129 號民事裁定,支付命令,給付貨款,,https://judgment.judicial.gov.tw/FJUD/data.asp...,26
